In [94]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from RNN import *
from collections import Counter
import torch.optim as optim


In [95]:
# stock = "AAPL"  # 根据需要更改股票代码
# train_df = pd.read_csv(f'/homes/Adam/DL_A3/us_data/train/{stock}.csv')
# test_df = pd.read_csv(f'/homes/Adam/DL_A3/us_data/test/{stock}.csv')

In [96]:

def read_and_combine_data(folder_path):
    all_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
    df_list = [pd.read_csv(file) for file in all_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df


# 定义训练和测试数据的文件夹路径
train_folder = '/homes/Adam/DL_A3/stock_train'
test_folder = '/homes/Adam/DL_A3/stock_test'

# 读取和合并训练数据
train_df = read_and_combine_data(train_folder)

# 读取和合并测试数据
test_df = read_and_combine_data(test_folder)

# 接下来，您可以对 train_df 和 test_df 进行进一步的处理和分析


In [97]:
sequence_length = 100

# Create detailed labels for training and testing datasets
train_detailed_labels = StockPricePredictor.create_labels(train_df['$close'],sequence_length)
test_detailed_labels = StockPricePredictor.create_labels(test_df['$close'],sequence_length)

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_detailed_labels)
test_labels_encoded = label_encoder.transform(test_detailed_labels)

# Select features
selected_features = ['$volume', 'daily_return', 'volatility', 'RSI', 'MA_diff']

X_train = train_df[selected_features].values
X_test = test_df[selected_features].values

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_sequences = StockPricePredictor.create_sequences(X_train_scaled, sequence_length)
X_test_sequences = StockPricePredictor.create_sequences(X_test_scaled, sequence_length)

# Adjust labels for sequences

y_train_sequences = train_labels_encoded[sequence_length:]
y_test_sequences = test_labels_encoded[sequence_length:]



# 分割原始训练数据集为新的训练集和验证集
X_train_new, X_val, y_train_new, y_val = train_test_split(
    X_train_sequences, y_train_sequences, test_size=0.2, random_state=42
)

# 转换为 PyTorch tensors
X_train_tensor = torch.tensor(X_train_new, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_new, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_sequences, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_sequences, dtype=torch.long)

# 创建 DataLoader 实例
train_data = TensorDataset(X_train_tensor, y_train_tensor)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [98]:

# 使用 Counter 来计数每个类别的出现次数
label_counts = Counter(train_detailed_labels)

# 打印每个类别的数量
for label, count in label_counts.items():
    print(f"Label '{label}': {count} occurrences")


Label 'Decrease': 9464 occurrences
Label 'Increase': 19692 occurrences


In [100]:
raise KeyError

KeyError: 

# RNN

In [ ]:
is_cuda_available = torch.cuda.is_available()
print("CUDA Available:", is_cuda_available)

# Model instantiation
input_size = len(selected_features)
hidden_size = 10
num_layers = 3
num_epochs = 100
dropout_rate = 0.1  # 确保这里的 dropout 率与您初始化模型时的一致

# Instantiate and move the model to GPU if available
rnn_model = StockRNN(input_size, hidden_size, num_layers, dropout_rate)
if is_cuda_available:
    rnn_model = rnn_model.cuda()

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.0005)

# Train the model
rnn_best_model_state, rnn_train_losses, rnn_val_losses, rnn_train_metrics, rnn_val_metrics = train_model(
    rnn_model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs
)

# Load the best model state
rnn_model.load_state_dict(rnn_best_model_state)

# Evaluate the performance of the trained model
rnn_model.eval()  # Set the model to evaluation mode
rnn_test_accuracy, rnn_test_loss, rnn_test_precision, rnn_test_recall, rnn_test_f1, rnn_test_auc_roc = evaluate_accuracy_and_loss(rnn_model, test_loader, criterion)

# Print evaluation results
print(f"RNN Test Accuracy: {rnn_test_accuracy:.2f}%, Loss: {rnn_test_loss:.4f}, Precision: {rnn_test_precision:.4f}, Recall: {rnn_test_recall:.4f}, F1 Score: {rnn_test_f1:.4f}, AUC-ROC: {rnn_test_auc_roc:.4f}")




CUDA Available: True
RNN Test Accuracy: 87.70%, Loss: 0.5291, Precision: 0.8881, Recall: 0.8900, F1 Score: 0.8891, AUC-ROC: 0.9128


In [ ]:


# print("Best Model Performance:")
# print(f"Accuracy: {best_accuracy:.2f}%, Loss: {best_loss:.4f}, Precision: {best_precision:.4f}, Recall: {best_recall:.4f}, F1 Score: {best_f1:.4f}, AUC-ROC: {best_auc_roc:.4f}")

# # 绘制损失图
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(rnn_train_losses, label='Train Loss')
# plt.plot(rnn_test_losses, label='Test Loss')
# plt.plot(rnn_val_losses, label='Validation Loss')  # 加入验证集损失的绘制
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Train vs Test vs Validation Loss')
# plt.legend()
# plt.show()


# LSTM

In [104]:
is_cuda_available = torch.cuda.is_available()
print("CUDA Available:", is_cuda_available)

# Model instantiation
input_size = len(selected_features)
hidden_size = 10
num_layers = 3
dropout_rate = 0.8  # 确保使用与训练时相同的 dropout 率

# 实例化模型并在有 GPU 的情况下移到 GPU
LSTM_model = StockLSTM(input_size, hidden_size, num_layers, dropout_rate)
if is_cuda_available:
    LSTM_model = LSTM_model.cuda()

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(LSTM_model.parameters(), lr=0.0005)

# 训练模型
num_epochs = 300
lstm_best_model_state, lstm_train_losses, lstm_val_losses, lstm_train_metrics, lstm_val_metrics = train_model(
    LSTM_model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs
)

# 加载最佳模型状态
LSTM_model.load_state_dict(lstm_best_model_state)

# 使用训练好的模型进行评估
LSTM_model.eval()  # Set the model to evaluation mode
lstm_test_accuracy, lstm_test_loss, lstm_test_precision, lstm_test_recall, lstm_test_f1, lstm_test_auc_roc = evaluate_accuracy_and_loss(LSTM_model, test_loader, criterion)

# 打印评估结果
print(f"LSTM Test Accuracy: {lstm_test_accuracy:.2f}%, Loss: {lstm_test_loss:.4f}, Precision: {lstm_test_precision:.4f}, Recall: {lstm_test_recall:.4f}, F1 Score: {lstm_test_f1:.4f}, AUC-ROC: {lstm_test_auc_roc:.4f}")


CUDA Available: True


LSTM Test Accuracy: 77.10%, Loss: 2.0240, Precision: 0.9027, Recall: 0.6887, F1 Score: 0.7814, AUC-ROC: 0.8422


# GRU


In [ ]:
is_cuda_available = torch.cuda.is_available()
print("CUDA Available:", is_cuda_available)

# Model instantiation
input_size = len(selected_features)
hidden_size = 10
num_layers = 3
dropout_rate = 0.1  # 确保使用与训练时相同的 dropout 率

# 实例化模型并在有 GPU 的情况下移到 GPU
GRU_model = StockGRU(input_size, hidden_size, num_layers, dropout_rate)
if is_cuda_available:
    GRU_model = GRU_model.cuda()

# 损失函数和优化器
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(GRU_model.parameters(), lr=0.0005)

# 训练模型
num_epochs = 200
gru_best_model_state, gru_train_losses, gru_val_losses, gru_train_metrics, gru_val_metrics = train_model(
    GRU_model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs
)

# 加载最佳模型状态
GRU_model.load_state_dict(gru_best_model_state)

# 使用训练好的模型进行评估
GRU_model.eval()  # Set the model to evaluation mode
gru_test_accuracy, gru_test_loss, gru_test_precision, gru_test_recall, gru_test_f1, gru_test_auc_roc = evaluate_accuracy_and_loss(GRU_model, test_loader, criterion)

# 打印评估结果
print(f"GRU Test Accuracy: {gru_test_accuracy:.2f}%, Loss: {gru_test_loss:.4f}, Precision: {gru_test_precision:.4f}, Recall: {gru_test_recall:.4f}, F1 Score: {gru_test_f1:.4f}, AUC-ROC: {gru_test_auc_roc:.4f}")


CUDA Available: True
GRU Test Accuracy: 76.98%, Loss: 1.6261, Precision: 0.8999, Recall: 0.6892, F1 Score: 0.7806, AUC-ROC: 0.8335


In [ ]:
# print("Best Model Performance:")
# print(f"Epoch: {gru_best_epoch_metrics[0]}, Accuracy: {gru_best_epoch_metrics[1]:.2f}%, Loss: {gru_best_epoch_metrics[2]:.4f}, Precision: {gru_best_epoch_metrics[3]:.4f}, Recall: {gru_best_epoch_metrics[4]:.4f}, F1 Score: {gru_best_epoch_metrics[5]:.4f}, AUC-ROC: {gru_best_epoch_metrics[6]:.4f}")

# # 绘制损失图
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(gru_train_losses, label='Train Loss')
# plt.plot(gru_test_losses, label='Test Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Train vs Test Loss')
# plt.legend()


# plt.show()